Plots for metabolites, saved at /home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_{metab}_results

Modify this line to plot before or after correction results

- sig_file = os.path.join(base_path, folder, fname, f'{contrast}', 'perm.th20.abs.sig.cluster.mgh')

In [ ]:
import os
import numpy as np
import nibabel as nib
from mayavi import mlab
from surfer import Brain
import matplotlib.pyplot as plt
import subprocess
import shutil


def setup_freesurfer():
    """Initialize FreeSurfer environment"""
    freesurfer_home = '/home/localadmin/freesurfer'
    subjects_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia"
    os.environ['FREESURFER_HOME'] = freesurfer_home
    os.environ['SUBJECTS_DIR'] = subjects_dir
    
    setup_cmd = f"bash -c 'source {freesurfer_home}/SetUpFreeSurfer.sh; env'"
    try:
        process = subprocess.Popen(setup_cmd, stdout=subprocess.PIPE, shell=True)
        output, _ = process.communicate()
        for line in output.decode().split('\n'):
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value
        print("FreeSurfer environment initialized successfully")
    except Exception as e:
        print(f"Error setting up FreeSurfer: {e}")
        raise


def is_valid_sig_file(sig_file):
    """Check if the p.mgh file contains valid, non-zero data."""
    if not os.path.exists(sig_file):
        return False
    try:
        data = nib.load(sig_file).get_fdata()
        if np.any(data > 0):  # Check for any non-zero values
            return True
    except Exception as e:
        print(f"Error reading file {sig_file}: {e}")
    return False


def create_brain_plot(subject_id, hemi, surf, sig_file, temp_file, view, threshold=1.3):
    """Create a brain plot for a single hemisphere and save it"""
    try:
        data = nib.load(sig_file).get_fdata().squeeze()

        brain = Brain(subject_id, hemi, surf, background="black", cortex="low_contrast", size=(600, 400))
        
        max_value = np.max(data)
        if max_value > threshold:
            brain.add_data(data,
                           min=threshold,
                           max=max_value,
                           mid=(threshold + max_value) / 2,
                           thresh=threshold,
                           colormap="YlOrRd",
                           alpha=1.0,
                           smoothing_steps=0,
                           colorbar=True,
                           remove_existing=True)
        else:
            print(f"No significant values in {sig_file}. Skipping plot.")
            return False
        
        # Render the specified view (lateral or medial)
        brain.show_view(view)
        brain.save_image(temp_file)
        mlab.close(brain._figures[0])
        return True
    except Exception as e:
        print(f"Error creating brain plot: {e}")
        return False


def plot_comparison(base_path, output_path, data_type, param_groups, contrasts, metab):
    """Generate and save diffusivity and kurtosis comparison plots"""
    temp_dir = os.path.join(output_path, 'temp')
    os.makedirs(temp_dir, exist_ok=True)
    views = ['lateral', 'medial']
    
    for param_group, param_list in param_groups.items():
        fig_rows = len(param_list)
        fig_cols = 24  
        fig = plt.figure(figsize=(3 * fig_cols, 5 * fig_rows))
        temp_images = []

        for i, param in enumerate(param_list):
            for j, contrast in enumerate(contrasts):
                for k, hemi in enumerate(['lh', 'rh']):
                    for l, view in enumerate(views):
                        col_index = j * 4 + k * 2 + l
                        folder = f'new_{metab}_results'
                        fname = f'group_{hemi}_{param}_{data_type}_demeaned_{contrast}.glmdir'
                        sig_file = os.path.join(base_path, folder, fname, f'{contrast}', 'perm.th20.abs.sig.cluster.mgh')
                        temp_file = os.path.join(temp_dir, f'temp_{data_type}_{param}_{contrast}_{hemi}_{view}.png')
                        temp_images.append(temp_file)

                        if is_valid_sig_file(sig_file):
                            success = create_brain_plot('fsaverage', hemi, 'inflated', sig_file, temp_file, view=view)
                            if not success:
                                print(f"Plot failed for {param}, {contrast}, {hemi}, {view}, {data_type}")
                        else:
                            print(f"Invalid or empty file: {sig_file}")

                        # Add the plot to the figure
                        ax = plt.subplot(fig_rows, fig_cols, i * fig_cols + col_index + 1)
                        if os.path.exists(temp_file):
                            img = plt.imread(temp_file)
                            ax.imshow(img)
                            ax.set_title(f"{hemi.upper()} {contrast} {view.capitalize()}", fontsize=8)
                        else:
                            ax.text(0.5, 0.5, 'File not found', ha='center', va='center')
                        ax.axis('off')

        plt.tight_layout()
        output_file = os.path.join(output_path, f"corrected_{data_type}_{param_group}.png")
        plt.savefig(output_file)
        plt.close()

        # Clean up temporary files
        for temp_file in temp_images:
            if os.path.exists(temp_file):
                os.remove(temp_file)

        # Remove the temporary directory
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)  # This will remove the directory and all its contents



def main():

    setup_freesurfer()
    
    param_groups = {
        "D": ['rd', 'md', 'ad', 'fa'],
        "K": ['mk', 'ak', 'rk']
    }
    contrasts = ['HC_metab_positive', 'EP_metab_positive', 'HC_metab_negative', 'EP_metab_negative', 'med_HCEP', 'med_EPHC']
    data_types = ['fwhm6', 'fwhm10']
    metabolites = ['glu','gsh', 'tcho', 'ins']
    
    for metab in metabolites:
        base_path = '/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia'
        output_path = os.path.join(base_path, f'new_{metab}_results')
        os.makedirs(output_path, exist_ok=True)

        for data_type in data_types:
            print(f"Processing data type: {data_type}")
            plot_comparison(base_path, output_path, data_type, param_groups, contrasts, metab)


if __name__ == "__main__":
    main()
